In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#cd /content/drive/My Drive/Colab Notebooks/SangamFinals/data

/content/drive/My Drive/Colab Notebooks/SangamFinals/data


In [0]:
"""!apt-get install libgeos-3.5.0
!apt-get install libgeos-dev
!pip install https://github.com/matplotlib/basemap/archive/master.zip
!pip install pyproj==1.9.6"""


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#from scipy.interpolate import griddata
#from mpl_toolkits.basemap import Basemap

### Replace chennai with gurgaon for gurgaon's data wherever necessary

In [0]:
df = pd.read_csv('chennai_data.csv', header=0, infer_datetime_format=True, parse_dates=['timestamp'], index_col=['timestamp'])

In [0]:
df.head()

,S.no,device_id,latitude,longitude,svrtime,humidity,temperature,heat_index,uv,luminence,ir,accx,accy,accz,acctemp,gyrox,gyroy,gyroz,so2_tmp,so2_gas,so2_conc,nh3,co,no2,c3h8,c4h10,ch4,h2,c2h5o4,pm01,pm25,pm10,pressure,draft,air_temp,air_speed
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-04-25 12:23:00,28630,M5,12.99430,80.2310,2019-04-25 12:23:00,44.5,26.8,80.49,0.02,261.0,256.0,-1168.0,436.0,16620.0,31.0,126.0,-1388.0,-128.0,27.0,1684.93,127.27,2.21,57.62,0.23,3377.23,1802.90,8701500.0,37.25,48.15,0.0,7.51168,17.33,27.87000,0.00,0.00,0.0
2019-04-25 12:26:00,28678,M5,12.99415,80.2305,2019-04-25 12:26:00,44.5,26.8,80.49,0.03,263.0,262.0,-1140.0,372.0,16656.0,31.0,128.0,-1381.0,-131.0,27.0,1690.59,117.59,2.35,66.14,0.22,3705.39,1950.59,14494000.0,45.98,57.73,0.0,7.51168,17.33,27.67000,0.00,0.00,0.0
2019-04-25 12:26:00,28683,M5,12.99400,80.2300,2019-04-25 12:26:00,45.0,26.8,80.54,0.03,263.0,261.0,-1076.0,432.0,16880.0,31.0,116.0,-1375.0,-120.0,27.0,1690.69,117.72,2.34,72.02,0.22,3662.95,1931.61,19857600.0,52.36,64.57,33.0,45.00000,60.00,7.52083,17.35,27.67,0.0
2019-04-25 12:26:00,28688,M5,12.99400,80.2300,2019-04-25 12:26:00,44.6,26.7,80.36,0.04,264.0,263.0,-1116.0,416.0,16520.0,31.0,129.0,-1380.0,-125.0,28.0,1691.02,116.27,2.30,65.79,0.22,3579.31,1894.09,14207500.0,45.60,57.32,34.0,47.00000,60.00,7.51168,17.33,27.67,0.0
2019-04-25 12:27:00,28692,M5,12.99390,80.2300,2019-04-25 12:27:00,44.7,26.8,80.51,0.04,262.0,260.0,-1144.0,464.0,16692.0,30.0,124.0,-1391.0,-136.0,27.0,1691.49,116.17,2.27,64.38,0.23,3497.28,1857.17,13112600.0,44.12,55.71,34.0,47.00000,59.00,7.51625,17.34,27.67,0.0


### Calculation of AQI
https://www.epa.vic.gov.au/your-environment/air/air-pollution/air-quality-index/calculating-a-station-air-quality-index

In [0]:
df.so2_conc = ((df.so2_conc) / 200) * 100
df.no2 = ((df.no2) / 120) * 100
df.co = ((df.co) / 9) * 100

In [0]:
df.drop(['S.no', 'svrtime'], axis=1, inplace=True)

In [0]:
hourly_groups = df.resample('1H')
eight_hourly = df.resample('8H')
twenty_four_hourly = df.resample('24H')

In [0]:
daily_groups = df.drop('device_id', axis=1).resample('H')
daily_groups_ = df.device_id.resample('H')

In [0]:
hourly_data = hourly_groups.mean()
eight_hourly_data = eight_hourly.mean()
twenty_four_hourly_data = twenty_four_hourly.mean()

In [0]:
hourly_data = hourly_data[['no2', 'so2_conc']]
eight_hourly_data = eight_hourly_data['co']
twenty_four_hourly_data = twenty_four_hourly_data[['pm25', 'pm10']]

In [0]:
hourly_data.reset_index(inplace=True)
eight_hourly_data['timestamp'] = eight_hourly_data.index
#eight_hourly_data.reset_index(inplace=True)
twenty_four_hourly_data.reset_index(inplace=True)

In [0]:
eight_hourly_data.timestamp = pd.to_datetime(eight_hourly_data.timestamp)

In [0]:
daily_data = daily_groups.apply(lambda array_like: np.nan if np.mean(array_like) == 0 else np.mean(array_like))
daily_data_ = daily_groups_.sum()
daily_data = pd.concat([daily_data, daily_data_], axis=1)
daily_data.shape

(1725, 34)

In [0]:
daily_data.interpolate(method="linear", inplace=True)
daily_data.air_speed.fillna(value=0, inplace=True)
frequent = np.argmax(daily_data.device_id.value_counts())
if frequent == 0:
    frequent = np.argmax(daily_data.device_id.value_counts()[1:])

#print(frequent)
daily_data.device_id = daily_data.device_id.apply(lambda x: frequent if x == 0 else x)
daily_data.device_id = daily_data.device_id.apply(lambda x: x[:2])
daily_data.head()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


,latitude,longitude,humidity,temperature,heat_index,uv,luminence,ir,accx,accy,accz,acctemp,gyrox,gyroy,gyroz,so2_tmp,so2_gas,so2_conc,nh3,co,no2,c3h8,c4h10,ch4,h2,c2h5o4,pm01,pm25,pm10,pressure,draft,air_temp,air_speed,device_id
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-04-25 12:00:00,12.991837,80.230203,45.628302,26.927358,80.791509,0.027453,261.867925,258.320755,-969.811321,463.660377,16718.830189,30.915094,122.150943,-1383.754717,-130.716981,27.018868,1694.564906,55.397736,2.132264,589.226415,0.202358,3197.179340,1719.753113,6.744742e+06,32.950755,43.252830,22.179245,30.684599,38.723302,8.479698,16.519528,26.402170,0.0,M5
2019-04-25 13:00:00,12.990755,80.230000,43.836715,28.627053,83.449372,0.022850,261.149758,255.473430,-494.492754,461.314010,16729.275362,32.859903,97.473430,-1373.183575,-124.048309,27.004831,1762.131884,-1.495990,2.301449,440.261406,0.192311,3582.105217,1895.009179,2.207624e+06,21.054251,29.423913,11.913043,16.340658,19.320483,7.615841,17.253816,29.613382,0.0,M5
2019-04-25 14:00:00,12.990833,80.230000,44.966351,29.672512,85.902607,0.023365,261.118483,254.289100,-479.810427,455.222749,16806.691943,34.033175,90.554502,-1356.293839,-123.156398,27.018957,1803.230379,-36.286422,2.582654,411.690890,0.186730,4262.230521,2196.083791,1.698445e+06,18.983791,26.925877,8.620853,12.085308,13.981043,7.514345,17.335829,30.689573,0.0,M5
2019-04-25 15:00:00,12.990757,80.230000,53.962037,30.696759,91.242639,0.022963,261.120370,254.138889,-478.481481,462.129630,16788.407407,34.981481,85.194444,-1351.759259,-119.569444,27.013889,1849.977917,-75.601806,2.945417,410.181584,0.171451,5197.144954,2599.613704,1.675400e+06,18.877361,26.795556,9.032407,12.555556,14.208333,7.515914,17.339259,31.612824,0.0,M5
2019-04-25 16:00:00,12.990694,80.230018,58.125806,31.019355,93.980876,0.023594,261.202765,254.133641,-470.082949,463.299539,16754.857143,35.000000,83.262673,-1352.580645,-121.345622,27.013825,1864.317097,-87.688134,3.040092,403.199181,0.172273,5453.214977,2708.389078,1.572036e+06,18.389862,26.192995,9.354839,13.290323,15.082949,7.517159,17.341982,31.955991,0.0,M5


In [0]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

daily_data.device_id = le.fit_transform(daily_data.device_id)

In [0]:
daily_data['aqi'] = 0

In [0]:
for i in daily_data.index:
    a = np.max(hourly_data.loc[hourly_data.timestamp <= i, 'no2'])
    b = np.max(hourly_data.loc[hourly_data.timestamp <= i, 'so2_conc'])
    d = np.max(twenty_four_hourly_data.loc[twenty_four_hourly_data.timestamp <= i, 'pm25'])
    e = np.max(twenty_four_hourly_data.loc[twenty_four_hourly_data.timestamp <= i, 'pm10'])
    daily_data.loc[i, 'aqi'] = max(a, b, d, e)

In [0]:
daily_data.head(10)

,latitude,longitude,humidity,temperature,heat_index,uv,luminence,ir,accx,accy,accz,acctemp,gyrox,gyroy,gyroz,so2_tmp,so2_gas,so2_conc,nh3,co,no2,c3h8,c4h10,ch4,h2,c2h5o4,pm01,pm25,pm10,pressure,draft,air_temp,air_speed,device_id,aqi
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-04-25 12:00:00,12.991837,80.230203,45.628302,26.927358,80.791509,0.027453,261.867925,258.320755,-969.811321,463.660377,16718.830189,30.915094,122.150943,-1383.754717,-130.716981,27.018868,1694.564906,55.397736,2.132264,589.226415,0.202358,3197.179340,1719.753113,6.744742e+06,32.950755,43.252830,22.179245,30.684599,38.723302,8.479698,16.519528,26.402170,0.0,0,55.397736
2019-04-25 13:00:00,12.990755,80.230000,43.836715,28.627053,83.449372,0.022850,261.149758,255.473430,-494.492754,461.314010,16729.275362,32.859903,97.473430,-1373.183575,-124.048309,27.004831,1762.131884,-1.495990,2.301449,440.261406,0.192311,3582.105217,1895.009179,2.207624e+06,21.054251,29.423913,11.913043,16.340658,19.320483,7.615841,17.253816,29.613382,0.0,0,55.397736
2019-04-25 14:00:00,12.990833,80.230000,44.966351,29.672512,85.902607,0.023365,261.118483,254.289100,-479.810427,455.222749,16806.691943,34.033175,90.554502,-1356.293839,-123.156398,27.018957,1803.230379,-36.286422,2.582654,411.690890,0.186730,4262.230521,2196.083791,1.698445e+06,18.983791,26.925877,8.620853,12.085308,13.981043,7.514345,17.335829,30.689573,0.0,0,55.397736
2019-04-25 15:00:00,12.990757,80.230000,53.962037,30.696759,91.242639,0.022963,261.120370,254.138889,-478.481481,462.129630,16788.407407,34.981481,85.194444,-1351.759259,-119.569444,27.013889,1849.977917,-75.601806,2.945417,410.181584,0.171451,5197.144954,2599.613704,1.675400e+06,18.877361,26.795556,9.032407,12.555556,14.208333,7.515914,17.339259,31.612824,0.0,0,55.397736
2019-04-25 16:00:00,12.990694,80.230018,58.125806,31.019355,93.980876,0.023594,261.202765,254.133641,-470.082949,463.299539,16754.857143,35.000000,83.262673,-1352.580645,-121.345622,27.013825,1864.317097,-87.688134,3.040092,403.199181,0.172273,5453.214977,2708.389078,1.572036e+06,18.389862,26.192995,9.354839,13.290323,15.082949,7.517159,17.341982,31.955991,0.0,0,55.397736
2019-04-25 17:00:00,12.990738,80.230000,62.189862,31.103226,96.151060,0.022212,261.000000,254.170507,-467.059908,462.617512,16737.032258,35.073733,89.276498,-1352.322581,-120.336406,27.013825,1870.184516,-92.722535,2.962258,394.122376,0.175960,5246.796359,2620.587742,1.445193e+06,17.761106,25.422627,11.963134,16.419355,18.525346,7.516020,17.339493,32.432673,0.0,0,55.397736
2019-04-25 18:00:00,12.990750,80.230000,64.222430,31.224766,97.661963,0.022383,261.018692,254.257009,-464.429907,460.392523,16718.392523,35.836449,88.060748,-1352.028037,-120.906542,27.009346,1873.817150,-95.586051,2.937897,389.959502,0.183022,5182.983178,2593.663738,1.389079e+06,17.473738,25.061729,12.266355,16.883178,19.294393,7.515439,17.338224,32.764673,0.0,0,55.397736
2019-04-25 19:00:00,12.990693,80.230002,62.550000,31.486239,97.916927,0.021376,260.729358,253.160550,-459.596330,449.633028,16707.761468,36.000000,89.252294,-1354.334862,-120.903670,27.009174,1882.164037,-102.753326,3.090138,391.111621,0.176491,5584.684037,2763.333761,1.404592e+06,17.552615,25.160734,9.885321,14.068807,16.321101,7.515790,17.338991,32.763761,0.0,0,55.397736
2019-04-25 20:00:00,12.990711,80.230000,61.246977,31.610233,97.762698,0.021302,260.725581,253.251163,-454.548837,449.469767,16727.200000,36.000000,86.432558,-1354.004651,-119.204651,27.004651,1886.614140,-106.528000,3.152279,390.840827,0.183295,5761.097163,2837.329023,1.400730e+06,17.533814,25.143395,10.767442,15.181395,17.418605,7.515444,17.338233,32.810233,0.0,0,55.397736


In [0]:
daily_data.corr()

,latitude,longitude,humidity,temperature,heat_index,uv,luminence,ir,accx,accy,accz,acctemp,gyrox,gyroy,gyroz,so2_tmp,so2_gas,so2_conc,nh3,co,no2,c3h8,c4h10,ch4,h2,c2h5o4,pm01,pm25,pm10,pressure,draft,air_temp,air_speed,device_id,aqi
latitude,1.000000,-0.811333,0.163920,0.103845,0.276921,0.106365,0.088791,0.110139,0.233188,-0.543188,-0.349730,0.096314,-0.089605,0.388782,0.093343,-0.052595,0.047402,0.124209,-0.038550,-0.385738,-0.018947,-0.592740,-0.581212,-0.004953,-0.183798,-0.553376,0.099924,0.177720,0.183034,-0.183307,-0.281403,-0.194442,0.536232,0.285593,0.746081
longitude,-0.811333,1.000000,-0.147226,-0.110858,-0.267703,-0.088152,-0.017030,-0.037773,-0.174043,0.322177,0.309204,-0.084709,0.038983,-0.315632,-0.078701,0.061227,-0.032314,-0.077983,0.017541,0.288935,0.016581,0.442323,0.427372,-0.005055,0.125165,0.413201,-0.108537,-0.188391,-0.194452,0.193827,0.278233,0.194857,-0.520670,-0.187120,-0.620048
humidity,0.163920,-0.147226,1.000000,-0.577987,0.206386,-0.327493,-0.270053,-0.325817,-0.191030,-0.039491,0.164204,-0.274227,-0.225475,-0.212872,0.145343,0.276288,-0.070169,-0.220319,-0.268568,-0.324651,0.117167,-0.168352,-0.178669,0.000258,-0.377324,-0.335155,-0.011361,-0.007199,-0.005147,-0.164928,0.103889,-0.007002,0.256697,-0.111328,0.271007
temperature,0.103845,-0.110858,-0.577987,1.000000,0.622737,0.048622,0.655909,0.663104,-0.193816,-0.069945,0.190320,-0.001682,-0.207016,-0.076346,-0.016991,0.057251,0.540678,0.186381,-0.061328,-0.092637,0.080615,-0.058952,-0.048216,-0.043491,-0.039578,-0.070954,0.115809,-0.118405,-0.110124,0.025597,0.190460,0.459742,-0.136945,-0.142436,-0.043526
heat_index,0.276921,-0.267703,0.206386,0.622737,1.000000,-0.097650,0.514924,0.490507,-0.317265,-0.220182,0.390555,-0.133532,-0.315440,-0.191115,0.004682,0.148717,0.436252,0.070425,-0.163718,-0.292208,0.081356,-0.233304,-0.233431,-0.041031,-0.266162,-0.331541,0.239876,-0.176994,-0.165120,-0.006350,0.371869,0.545580,0.116781,-0.321431,0.248559
uv,0.106365,-0.088152,-0.327493,0.048622,-0.097650,1.000000,-0.087340,0.020912,0.336249,-0.308474,-0.025440,0.969205,0.519932,0.283553,-0.851558,-0.806948,-0.683029,0.262885,0.588175,0.314054,-0.238871,-0.065304,-0.064482,-0.014448,0.879806,0.196564,0.265101,-0.073390,-0.073254,0.710596,0.111400,0.017822,0.036455,-0.108485,0.095805
luminence,0.088791,-0.017030,-0.270053,0.655909,0.514924,-0.087340,1.000000,0.986850,-0.302148,-0.179837,0.344212,-0.180269,-0.315907,-0.140521,0.081626,0.163278,0.479338,0.234413,-0.121675,-0.090600,0.048376,-0.030623,-0.029474,-0.023435,-0.180984,-0.071759,0.226152,-0.059524,-0.047353,0.025083,0.302426,0.510625,-0.346761,-0.214025,-0.128697
ir,0.110139,-0.037773,-0.325817,0.663104,0.490507,0.020912,0.986850,1.000000,-0.253759,-0.217556,0.318461,-0.076622,-0.253067,-0.089233,0.001076,0.069226,0.402924,0.294441,-0.052178,-0.049088,0.018094,-0.039879,-0.037722,-0.026701,-0.085662,-0.046265,0.242106,-0.080429,-0.068539,0.086308,0.297969,0.495202,-0.334270,-0.200732,-0.111445
accx,0.233188,-0.174043,-0.191030,-0.193816,-0.317265,0.336249,-0.302148,-0.253759,1.000000,-0.531105,-0.761434,0.333941,0.224038,0.716126,0.046715,-0.313773,-0.290164,0.073253,0.259896,-0.045927,-0.121520,-0.230036,-0.249840,0.006849,0.185408,-0.190120,-0.208105,0.244431,0.237303,-0.226911,-0.703420,-0.718964,0.573001,0.526465,0.539838
accy,-0.543188,0.322177,-0.039491,-0.069945,-0.220182,-0.308474,-0.179837,-0.217556,-0.531105,1.000000,0.274780,-0.290757,0.171133,-0.501336,0.087641,0.203216,0.096711,-0.210128,-0.069228,0.309923,0.042162,0.380963,0.409531,0.020122,-0.015578,0.460607,-0.021769,-0.067431,-0.073382,0.023976,0.250419,0.192774,-0.383271,-0.236721,-0.621275


In [0]:
daily_data.drop(['air_speed'], axis=1, inplace=True)

In [0]:
daily_data.interpolate(method="linear", inplace=True)

In [0]:
daily_data.head()

,latitude,longitude,humidity,temperature,heat_index,uv,luminence,ir,accx,accy,accz,acctemp,gyrox,gyroy,gyroz,so2_tmp,so2_gas,so2_conc,nh3,co,no2,c3h8,c4h10,ch4,h2,c2h5o4,pm01,pm25,pm10,pressure,draft,air_temp,device_id,aqi
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-04-25 12:00:00,12.991837,80.230203,45.628302,26.927358,80.791509,0.027453,261.867925,258.320755,-969.811321,463.660377,16718.830189,30.915094,122.150943,-1383.754717,-130.716981,27.018868,1694.564906,55.397736,2.132264,589.226415,0.202358,3197.179340,1719.753113,6.744742e+06,32.950755,43.252830,22.179245,30.684599,38.723302,8.479698,16.519528,26.402170,0,55.397736
2019-04-25 13:00:00,12.990755,80.230000,43.836715,28.627053,83.449372,0.022850,261.149758,255.473430,-494.492754,461.314010,16729.275362,32.859903,97.473430,-1373.183575,-124.048309,27.004831,1762.131884,-1.495990,2.301449,440.261406,0.192311,3582.105217,1895.009179,2.207624e+06,21.054251,29.423913,11.913043,16.340658,19.320483,7.615841,17.253816,29.613382,0,55.397736
2019-04-25 14:00:00,12.990833,80.230000,44.966351,29.672512,85.902607,0.023365,261.118483,254.289100,-479.810427,455.222749,16806.691943,34.033175,90.554502,-1356.293839,-123.156398,27.018957,1803.230379,-36.286422,2.582654,411.690890,0.186730,4262.230521,2196.083791,1.698445e+06,18.983791,26.925877,8.620853,12.085308,13.981043,7.514345,17.335829,30.689573,0,55.397736
2019-04-25 15:00:00,12.990757,80.230000,53.962037,30.696759,91.242639,0.022963,261.120370,254.138889,-478.481481,462.129630,16788.407407,34.981481,85.194444,-1351.759259,-119.569444,27.013889,1849.977917,-75.601806,2.945417,410.181584,0.171451,5197.144954,2599.613704,1.675400e+06,18.877361,26.795556,9.032407,12.555556,14.208333,7.515914,17.339259,31.612824,0,55.397736
2019-04-25 16:00:00,12.990694,80.230018,58.125806,31.019355,93.980876,0.023594,261.202765,254.133641,-470.082949,463.299539,16754.857143,35.000000,83.262673,-1352.580645,-121.345622,27.013825,1864.317097,-87.688134,3.040092,403.199181,0.172273,5453.214977,2708.389078,1.572036e+06,18.389862,26.192995,9.354839,13.290323,15.082949,7.517159,17.341982,31.955991,0,55.397736


In [0]:
daily_data.reset_index(inplace=True)

In [0]:
daily_data.head()

,timestamp,latitude,longitude,humidity,temperature,heat_index,uv,luminence,ir,accx,accy,accz,acctemp,gyrox,gyroy,gyroz,so2_tmp,so2_gas,so2_conc,nh3,co,no2,c3h8,c4h10,ch4,h2,c2h5o4,pm01,pm25,pm10,pressure,draft,air_temp,device_id,aqi
0,2019-04-25 12:00:00,12.991837,80.230203,45.628302,26.927358,80.791509,0.027453,261.867925,258.320755,-969.811321,463.660377,16718.830189,30.915094,122.150943,-1383.754717,-130.716981,27.018868,1694.564906,55.397736,2.132264,589.226415,0.202358,3197.179340,1719.753113,6.744742e+06,32.950755,43.252830,22.179245,30.684599,38.723302,8.479698,16.519528,26.402170,0,55.397736
1,2019-04-25 13:00:00,12.990755,80.230000,43.836715,28.627053,83.449372,0.022850,261.149758,255.473430,-494.492754,461.314010,16729.275362,32.859903,97.473430,-1373.183575,-124.048309,27.004831,1762.131884,-1.495990,2.301449,440.261406,0.192311,3582.105217,1895.009179,2.207624e+06,21.054251,29.423913,11.913043,16.340658,19.320483,7.615841,17.253816,29.613382,0,55.397736
2,2019-04-25 14:00:00,12.990833,80.230000,44.966351,29.672512,85.902607,0.023365,261.118483,254.289100,-479.810427,455.222749,16806.691943,34.033175,90.554502,-1356.293839,-123.156398,27.018957,1803.230379,-36.286422,2.582654,411.690890,0.186730,4262.230521,2196.083791,1.698445e+06,18.983791,26.925877,8.620853,12.085308,13.981043,7.514345,17.335829,30.689573,0,55.397736
3,2019-04-25 15:00:00,12.990757,80.230000,53.962037,30.696759,91.242639,0.022963,261.120370,254.138889,-478.481481,462.129630,16788.407407,34.981481,85.194444,-1351.759259,-119.569444,27.013889,1849.977917,-75.601806,2.945417,410.181584,0.171451,5197.144954,2599.613704,1.675400e+06,18.877361,26.795556,9.032407,12.555556,14.208333,7.515914,17.339259,31.612824,0,55.397736
4,2019-04-25 16:00:00,12.990694,80.230018,58.125806,31.019355,93.980876,0.023594,261.202765,254.133641,-470.082949,463.299539,16754.857143,35.000000,83.262673,-1352.580645,-121.345622,27.013825,1864.317097,-87.688134,3.040092,403.199181,0.172273,5453.214977,2708.389078,1.572036e+06,18.389862,26.192995,9.354839,13.290323,15.082949,7.517159,17.341982,31.955991,0,55.397736


In [0]:
daily_data = daily_data[10:]

In [0]:
daily_data['hour'] = daily_data.timestamp.dt.hour
daily_data['H'] = 15  * (daily_data.hour - 12)
Z = (np.sin(daily_data.latitude)*np.sin(daily_data.longitude) + np.cos(daily_data.latitude)* np.cos(daily_data.longitude)*np.cos(daily_data.H))
s =  1000 * np.cos(Z)
daily_data['Solar'] = s

In [0]:
daily_data.head()

,timestamp,latitude,longitude,humidity,temperature,heat_index,uv,luminence,ir,accx,accy,accz,acctemp,gyrox,gyroy,gyroz,so2_tmp,so2_gas,so2_conc,nh3,co,no2,c3h8,c4h10,ch4,h2,c2h5o4,pm01,pm25,pm10,pressure,draft,air_temp,device_id,aqi,hour,H,Solar
10,2019-04-25 22:00:00,12.990657,80.230003,60.610648,31.812500,98.274815,0.021250,260.680556,253.203704,-464.759259,452.685185,16727.537037,36.0,86.018519,-1351.425926,-120.166667,27.000000,1894.873148,-113.581690,3.243981,392.354424,0.183333,6008.258056,2940.363750,1.421009e+06,17.638380,25.272222,12.712963,17.708333,20.250000,7.515406,17.338148,33.042130,0,55.397736,22,150,945.118595
11,2019-04-25 23:00:00,12.990768,80.230000,60.973853,31.813761,98.473807,0.020917,260.720183,253.275229,-457.486239,453.247706,16733.669725,36.0,86.779817,-1353.706422,-118.954128,27.004587,1895.037706,-113.538234,3.258028,388.120285,0.187997,6043.727523,2955.222064,1.365129e+06,17.350826,24.909037,14.110092,19.463303,22.633028,7.515455,17.338257,33.050000,0,55.397736,23,165,914.696351
12,2019-04-26 00:00:00,12.990743,80.230000,61.439535,31.856744,98.910326,0.020279,260.562791,253.962791,-464.316279,453.041860,16736.725581,36.0,85.665116,-1352.320930,-119.981395,27.004651,1895.846605,-114.316070,3.255721,384.677519,0.191667,6036.430791,2952.570930,1.321526e+06,17.117953,24.624000,16.744186,23.046512,27.004651,7.514955,17.337163,33.066279,0,55.397736,0,-180,889.852411
13,2019-04-26 01:00:00,12.990719,80.230009,61.507834,31.922581,99.229355,0.021935,260.926267,256.161290,-453.917051,443.705069,16733.935484,36.0,84.622120,-1352.700461,-120.230415,27.004608,1899.207281,-117.061198,3.274286,382.484383,0.191667,6091.654747,2974.979263,1.293724e+06,16.968249,24.441705,18.548387,25.410138,29.981567,7.514903,17.337051,33.153318,0,55.397736,1,-165,914.714162
14,2019-04-26 02:00:00,12.990768,80.230019,60.942791,32.062326,99.519767,0.023721,261.265116,263.297674,-455.423256,450.865116,16722.716279,36.0,85.832558,-1352.855814,-119.432558,27.013953,1905.008279,-122.130186,3.299581,380.260982,0.191667,6170.009581,3007.569256,1.265800e+06,16.814651,24.253070,18.134884,24.804651,29.283721,7.514997,17.337256,33.232233,0,55.397736,2,-150,945.087895


In [0]:
#daily_data.to_csv("gurgaon_data_solar.csv", index=False)

In [0]:
daily_data.corr()[-4:]

,latitude,longitude,humidity,temperature,heat_index,uv,luminence,ir,accx,accy,accz,acctemp,gyrox,gyroy,gyroz,so2_tmp,so2_gas,so2_conc,nh3,co,no2,c3h8,c4h10,ch4,h2,c2h5o4,pm01,pm25,pm10,pressure,draft,air_temp,device_id,aqi,hour,H,Solar
aqi,0.739647,-0.609401,0.267852,-0.056300,0.237321,0.094284,-0.136579,-0.119666,0.561750,-0.632785,-0.578546,0.106333,-0.070038,0.561669,0.177194,-0.069549,-0.020250,0.031785,-0.031938,-0.461859,-0.025728,-0.583279,-0.586430,-0.004135,-0.227089,-0.661917,-0.114852,0.221934,0.220662,-0.342005,-0.522272,-0.462744,0.413499,1.000000,0.002177,0.002177,-0.380188
hour,-0.030712,0.024831,0.323426,-0.368771,-0.184022,-0.041181,-0.300959,-0.322326,-0.027118,0.045254,-0.002183,-0.008948,0.025510,-0.017263,-0.013218,-0.000138,-0.198590,-0.108060,-0.013287,0.000769,0.000839,-0.015177,-0.000026,0.003429,-0.008366,0.001019,-0.084662,-0.004240,-0.007992,-0.000450,-0.005469,-0.095866,0.014294,0.002177,1.000000,1.000000,0.070723
H,-0.030712,0.024831,0.323426,-0.368771,-0.184022,-0.041181,-0.300959,-0.322326,-0.027118,0.045254,-0.002183,-0.008948,0.025510,-0.017263,-0.013218,-0.000138,-0.198590,-0.108060,-0.013287,0.000769,0.000839,-0.015177,-0.000026,0.003429,-0.008366,0.001019,-0.084662,-0.004240,-0.007992,-0.000450,-0.005469,-0.095866,0.014294,0.002177,1.000000,1.000000,0.070723
Solar,-0.513599,0.412731,-0.088756,-0.032104,-0.128104,-0.056488,-0.040175,-0.051602,-0.136841,0.291407,0.177388,-0.055023,0.052312,-0.194903,-0.041897,0.021659,-0.010104,-0.065187,0.031315,0.203155,0.005046,0.311373,0.317737,-0.005947,0.092191,0.288785,-0.041775,-0.088226,-0.090609,0.091888,0.144113,0.103611,-0.133753,-0.380188,0.070723,0.070723,1.000000


In [0]:
daily_data.drop(['hour', 'H'], axis=1, inplace=True)

In [0]:
daily_data.isnull().sum()

timestamp      0
latitude       0
longitude      0
humidity       0
temperature    0
heat_index     0
uv             0
luminence      0
ir             0
accx           0
accy           0
accz           0
acctemp        0
gyrox          0
gyroy          0
gyroz          0
so2_tmp        0
so2_gas        0
so2_conc       0
nh3            0
co             0
no2            0
c3h8           0
c4h10          0
ch4            0
h2             0
c2h5o4         0
pm01           0
pm25           0
pm10           0
pressure       0
draft          0
air_temp       0
device_id      0
aqi            0
Solar          0
dtype: int64

In [0]:
#daily_data.to_csv("chennai_data_aqi_hourly.csv", index=False)

In [0]:
y_train = daily_data.pop('aqi')
x_train = daily_data.drop(['timestamp'], axis=1)

In [0]:
daily_data['aqi'] = y_train

### Training XGBRegressor for spatial interpolation of AQI i.e. outside gurgaon or chennai

In [0]:
from xgboost import XGBRegressor

model = XGBRegressor()

model.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[05:19:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [0]:
model.score(x_train, y_train)

0.9918912968513192

In [0]:
y_pred = model.predict(x_train)
y_pred = pd.DataFrame(y_pred, columns=['y_pred'])

In [0]:
#temp = pd.concat([daily_data, y_pred], axis=1)
#temp.to_csv("complete_dataset_with_preds_chennai.csv", index=False)